In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd

from torch.utils.data import Dataset, DataLoader

from utils import load_test_data, visualize_test_region, generate_training_samples
from dataset import STDataset
from baseline import RandomRegionBaseline, TissueSpecificRandomRegionBaseline
from evaluate import Evaluator

In [6]:
all_test_items = load_test_data(num_holes=10)

c:\Docs_and_Stuff\Emory\Lab\DISCO\DISCO\utils.py:54: SyntaxWarning: invalid escape sequence '\D'
  fold_dir = "C:\Docs_and_Stuff\Emory\Lab\DISCO\DISCO"


Seeding all randomness with seed=2024
Donor_id: MsBrainAgingSpatialDonor_1
Slice_id: 0


c:\Docs_and_Stuff\Emory\Lab\DISCO\DISCO\utils.py:80: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  fov_boundaries = slice_obs_df.groupby('fov').agg(


Donor_id: MsBrainAgingSpatialDonor_2
Slice_id: 0


c:\Docs_and_Stuff\Emory\Lab\DISCO\DISCO\utils.py:80: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  fov_boundaries = slice_obs_df.groupby('fov').agg(


...

c:\Docs_and_Stuff\Emory\Lab\DISCO\DISCO\utils.py:80: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  fov_boundaries = slice_obs_df.groupby('fov').agg(


Donor_id: MsBrainAgingSpatialDonor_12
Slice_id: 0


c:\Docs_and_Stuff\Emory\Lab\DISCO\DISCO\utils.py:80: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  fov_boundaries = slice_obs_df.groupby('fov').agg(


Slice_id: 1


c:\Docs_and_Stuff\Emory\Lab\DISCO\DISCO\utils.py:80: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  fov_boundaries = slice_obs_df.groupby('fov').agg(


In [7]:
# Initialize dictionaries to store metrics for both baselines
metrics = {
    'RandomRegionBaseline': {
        'mse': [],
        'f1': [],
        'cosine_sim': [],
        'chamfer_dist': [],
        'emd': []
    },
    'TissueSpecificRandomRegionBaseline': {
        'mse': [],
        'f1': [],
        'cosine_sim': [],
        'chamfer_dist': [],
        'emd': []
    }
}

In [14]:
for i, test_item in enumerate(all_test_items):
    print(f"Test Area {i+1}:")
    print(f"  Dominant Tissue: {test_item.test_area.dominant_tissue}")
    print(f"  Number of cells in ground truth: {len(test_item.ground_truth.hole_cells)}")
    
    # Apply RandomRegionBaseline
    random_baseline = RandomRegionBaseline(test_item.adata, test_item.test_area)
    random_coords, random_gene_expressions = random_baseline.fill_region()

    # Apply TissueSpecificRandomRegionBaseline
    tissue_baseline = TissueSpecificRandomRegionBaseline(test_item.adata, test_item.test_area)
    tissue_coords, tissue_gene_expressions = tissue_baseline.fill_region()

    # Evaluate predictions for RandomRegionBaseline
    true_coords = test_item.ground_truth.hole_cells[['center_x', 'center_y']].values
    true_gene_expressions = test_item.ground_truth.gene_expression

    mse_r, f1_r, cosine_sim_r = Evaluator.evaluate_expression(true_coords, true_gene_expressions, random_coords, random_gene_expressions)
    chamfer_dist_r = Evaluator.chamfer_distance(true_coords, random_coords)
    emd_r = Evaluator.calculate_emd(true_coords, random_coords)

    # Evaluate predictions for TissueSpecificRandomRegionBaseline
    mse_t, f1_t, cosine_sim_t = Evaluator.evaluate_expression(true_coords, true_gene_expressions, tissue_coords, tissue_gene_expressions)
    chamfer_dist_t = Evaluator.chamfer_distance(true_coords, tissue_coords)
    emd_t = Evaluator.calculate_emd(true_coords, tissue_coords)

    # Collect results for RandomRegionBaseline
    metrics['RandomRegionBaseline']['mse'].append(mse_r)
    metrics['RandomRegionBaseline']['f1'].append(f1_r)
    metrics['RandomRegionBaseline']['cosine_sim'].append(cosine_sim_r)
    metrics['RandomRegionBaseline']['chamfer_dist'].append(chamfer_dist_r)
    metrics['RandomRegionBaseline']['emd'].append(emd_r)

    # Collect results for TissueSpecificRandomRegionBaseline
    metrics['TissueSpecificRandomRegionBaseline']['mse'].append(mse_t)
    metrics['TissueSpecificRandomRegionBaseline']['f1'].append(f1_t)
    metrics['TissueSpecificRandomRegionBaseline']['cosine_sim'].append(cosine_sim_t)
    metrics['TissueSpecificRandomRegionBaseline']['chamfer_dist'].append(chamfer_dist_t)
    metrics['TissueSpecificRandomRegionBaseline']['emd'].append(emd_t)

Test Area 1:
  Dominant Tissue: striatum
  Number of cells in ground truth: 50
[[-4327.32420612  1836.21752937]
 [-4232.1277036   1890.10303289]
 [-4336.91620589  1789.12953051]
 [-4208.85620416  1855.00503373]
 [-4201.98920432  1844.81353398]
 [-4178.1182049   1807.55053006]
 [-4260.63120291  1875.27903325]
 [-4176.59220493  1876.04203323]
 [-4306.39620662  1898.93203268]
 [-4275.71270736  1779.15603075]
 [-4311.24670651  1784.87853061]
 [-4317.29620636  1829.35052954]
 [-4284.37820715  1762.09753116]
 [-4317.07820637  1896.31603274]
 [-4155.17370545  1811.09302998]
 [-4195.88520447  1861.10903359]
 [-4266.77470758  1821.55702973]
 [-4300.51020676  1865.08753349]
 [-4304.54320667  1875.44253324]
 [-4336.5347059   1838.77902931]
 [-4257.90620298  1827.27952959]
 [-4241.06570338  1848.20752908]
 [-4194.84970449  1793.7075304 ]
 [-4236.65120349  1815.88902986]
 [-4330.48520604  1869.77453338]
 [-4299.91070678  1882.74553307]
 [-4286.28570711  1886.45153298]
 [-4279.14620728  1798.6670302

In [6]:
for method in metrics:
    print(f"Results for {method}:")
    for metric in metrics[method]:
        mean_value = np.mean(metrics[method][metric])
        std_value = np.std(metrics[method][metric])
        print(f"  {metric.capitalize()}: Mean = {mean_value:.4f}, Std = {std_value:.4f}")

Results for RandomRegionBaseline:
  Mse: Mean = 1.9030, Std = 0.2837
  F1: Mean = 0.8143, Std = 0.0359
  Cosine_sim: Mean = 0.0329, Std = 0.0402
  Chamfer_dist: Mean = 26.7087, Std = 3.0385
  Emd: Mean = 21.8310, Std = 3.6684
Results for TissueSpecificRandomRegionBaseline:
  Mse: Mean = 1.8188, Std = 0.3385
  F1: Mean = 0.8236, Std = 0.0366
  Cosine_sim: Mean = 0.0725, Std = 0.0544
  Chamfer_dist: Mean = 27.0084, Std = 3.2562
  Emd: Mean = 22.2538, Std = 4.1351


In [10]:
training_samples = generate_training_samples(num_samples_per_slice=3)

c:\Docs_and_Stuff\Emory\Lab\DISCO\DISCO\utils.py:54: SyntaxWarning: invalid escape sequence '\D'
  fold_dir = "C:\Docs_and_Stuff\Emory\Lab\DISCO\DISCO"
c:\Docs_and_Stuff\Emory\Lab\DISCO\DISCO\utils.py:232: SyntaxWarning: invalid escape sequence '\D'
  fold_dir = "/extra/zhanglab0/SpatialTranscriptomicsData/"


Seeding all randomness with seed=2024


In [11]:
dataset = STDataset(training_samples)

In [12]:
# Define a DataLoader to handle batching
dataloader = DataLoader(dataset, batch_size=10, shuffle=True, num_workers=4)

# Example of iterating over the DataLoader
for batch in dataloader:
    positions = batch['positions']
    expressions = batch['expressions']
    metadata = batch['metadata']
    
    # Use the positions, expressions for model training
    # The metadata could be used for logging, tracking, or conditioning if needed
    print(positions.shape, expressions.shape)

torch.Size([10, 50, 2]) torch.Size([10, 50, 374])
